In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F

import matplotlib.pyplot as plt 
from torch.utils.data import Dataset, DataLoader

In [2]:
from pathlib import Path 
from typing import Tuple
from torchvision.io import read_image

root_folder = "./data/MelanomaDetectionLabeled/"

# setup transform with Imagenet normalization parameters
transform_test = transforms.Compose([
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

transform_train = transforms.Compose([
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), 
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(30),
])

In [3]:
# Test image load 
test_img_path = root_folder + "labeled/0_0.jpg"
im = read_image(test_img_path).type(torch.float32)/255.0
im.shape, im.dtype

(torch.Size([3, 32, 32]), torch.float32)

In [4]:

class MelanomaDataset(Dataset):
    def __init__(self, folder, transform=None):
        super().__init__()
        self.folder = Path(folder).resolve()
        self.transform = transform
        self.img_path_list = list(self.folder.glob('*.jpg'))
        
        
    def __len__(self):
        return len(self.img_path_list)
    
    def __getitem__(self, index) -> Tuple:
        img_path = self.img_path_list[index]
        img = read_image(str(img_path)).type(torch.float32)/255.0        
        if self.transform:
            img = self.transform(img)
        label = img_path.stem.split('_')[1]
        return img, int(label) 
    
    
# test 

ds_train = MelanomaDataset(root_folder + "labeled", transform=transform_train)
ds_test = MelanomaDataset(root_folder + "test", transform=transform_test)

print("Train dataset size: ", len(ds_train))
print("Test dataset size: ", len(ds_test))

Train dataset size:  200
Test dataset size:  600


In [5]:
# Data Loader for Train & Test 
dl_train = DataLoader(ds_train, batch_size=64, shuffle=True)
dl_test = DataLoader(ds_train, batch_size=64, shuffle=False)

print("Train Loader Check:")
for X, y in dl_train:
    print(f"Shape of X [N, C, H, W]: {X.shape}, X.dtype: {X.dtype}")
    print(f"Shape of y: {y.shape}, y.dtype: {y.dtype}")
    break

print("\nTest Loader Check:")
for X, y in dl_test:
    print(f"Shape of X [N, C, H, W]: {X.shape}, X.dtype: {X.dtype}")
    print(f"Shape of y: {y.shape}, y.dtype: {y.dtype}")
    break

Train Loader Check:
Shape of X [N, C, H, W]: torch.Size([64, 3, 32, 32]), X.dtype: torch.float32
Shape of y: torch.Size([64]), y.dtype: torch.int64

Test Loader Check:
Shape of X [N, C, H, W]: torch.Size([64, 3, 32, 32]), X.dtype: torch.float32
Shape of y: torch.Size([64]), y.dtype: torch.int64


In [ ]:
class Net(nn.Module):
    def __init__(self, img_shape=(3, 32, 32)):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout(0.5)

        dummy = torch.zeros(img_shape).unsqueeze_(0)
        flatten_size = self.cnn_stack(dummy).shape[1]
        self.fc1 = nn.Linear(flatten_size, 128)
        self.fc2 = nn.Linear(128, 2)

    def cnn_stack(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        return x

    def forward(self, x):
        x = self.cnn_stack(x)

        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        return self.fc2(x)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Current Device: ", device)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=0.2)
scheduler = StepLR(optimizer, step_size=7, gamma=0.8)
loss_fn = nn.CrossEntropyLoss()